# Import libraries and data

In [1]:
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [6]:
# Get annotated tweets
tweets = pd.read_csv('../data/local/labelled_tweets_clean_all_mturk_agg.csv').reset_index()
tweets.head()

index                                               text  neutral  \
0      0  "#Shingrix Vaccine Against Shingles Remains In...        3   
1      1  "...French scientist Louis Pasteur coined the ...        3   
2      2  "...due to potentially unexplained illnesses."...        2   
3      3  "...on average, it takes 10.7 years to develop...        3   
4      4  "..only 29 percent of the teens its members in...        3   

   negative  positive  total  
0         0         0      3  
1         0         0      3  
2         1         7     10  
3         0         0      3  
4         0         0      3

In [77]:
# Get annotated tweets
tweets_1000 = pd.read_csv('../data/local/labelled_tweets_clean_all_mturk_agg_sample1000.csv').reset_index()
tweets_1000.head()

index                                               text  neutral  \
0      0  @user Deploy the J&J one dose vaccine to all c...        3   
1      1  There's a petition on <url> that's requesting ...        4   
2      2  This week: #outbreaks in #China, #Nigeria & #S...        3   
3      3  #ChristmasPuns How does Darth Vader like his C...        3   
4      4  Colombia to participate in phase three vaccine...        2   

   negative  positive  total agree_mturk  id_tweets  
0         0         0      3     neutral        515  
1         0         0      4     neutral       3833  
2         0         0      3     neutral       3908  
3         0         0      3     neutral        103  
4         1         8     11    positive       1282

# Using sample to test prompt

In [83]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user of the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [95]:
prompt_raw_0 = prompt_raw
prompt_raw_0

'Give me the sentiment regarding vaccination in tweet text delimited by triple backticks. Use one of the following words: neutral, negative, positive. ```{TWEET_TEXT_HERE}```'

In [82]:
prompt_raw_1.write('../data/local/gpt_prompt_1.csv')


AttributeError: 'str' object has no attribute 'write'

In [74]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [78]:
len(tweets_1000.index)

1000

In [79]:
tweets_1000.iloc[1,1]

"There's a petition on <url> that's requesting Ndemic Creations to add the Anti-Vaxxer Movement as a mechanic in Plague Inc. 🤣 <url>"

In [101]:
id_gpt = []
sent_gpt = []

In [102]:
for i in range(0,1000):
    prompt_i = prompt_raw_1.replace('TWEET_TEXT_HERE', tweets_1000.iloc[i,1])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets_1000.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [87]:
prompt = np.array([1])

In [103]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         1000 non-null   int64 
 1   sentiment_gpt  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [104]:
df_gpt_total['prompt'] = 1

In [105]:
df_gpt_total.tail()

id_gpt                                      sentiment_gpt  prompt
995     995  \n\nNeutral. The sentiment expressed in the tw...       1
996     996  \n\nNeutral. This tweet is a question and does...       1
997     997  \n\nNeutral. The sentiment in this tweet is ne...       1
998     998  \n\nNeutral. This tweet is simply discussing i...       1
999     999  \n\nNegative. The sentiment expressed in the t...       1

In [59]:
#df_gpt_total_all = df_gpt_total.append(df_gpt_total2)

C:\Users\lespinosa\AppData\Local\Temp\ipykernel_23560\3678373193.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gpt_total_all = df_gpt_total.append(df_gpt_total2)


In [107]:
df_gpt_total.to_csv('../data/local/gpt_sentiment_mturk_prompt1.csv')


# Using three categories for vaccine sentiment

In [10]:
# Prompt for GPT
prompt_raw = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. \
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [11]:
len(tweets.index)

4786

In [12]:
test = tweets.iloc[:101, :]
test.info

<bound method DataFrame.info of      index                                               text  neutral  \
0        0  "#Shingrix Vaccine Against Shingles Remains In...        3   
1        1  "...French scientist Louis Pasteur coined the ...        3   
2        2  "...due to potentially unexplained illnesses."...        2   
3        3  "...on average, it takes 10.7 years to develop...        3   
4        4  "..only 29 percent of the teens its members in...        3   
..     ...                                                ...      ...   
96      96  #COVID19 vaccine could be delivered in mailabl...        4   
97      97  #COVID19Vaccine in 1 month? CSIR pins hope on ...        4   
98      98  #COVID19dz Ignorance at the utmost level as co...        1   
99      99  #CRO Vaccine Market in Austria to 2021 Market ...        3   
100    100  #California has just implemented one of the #s...        3   

     negative  positive  total  
0           0         0      3  
1           0

In [7]:
tweets1 = tweets.iloc[0:306,:]
tweets2 = tweets.iloc[306:601,:]
tweets3 = tweets.iloc[601:901, :]
tweets4 = tweets.iloc[901:1201, :]


In [14]:
test.iloc[1,1]

'"...French scientist Louis Pasteur coined the term “vaccination” in honour of Jenner’s work (vacca being the Latin for cow)" - TIL! 🐮💉 <url>'

In [68]:
id_gpt = []
sent_gpt = []

In [69]:
for i in range(0,4785):
    prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[i,1])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [38]:
run = np.array([1])

In [70]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4785 entries, 0 to 4784
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         4785 non-null   int64 
 1   sentiment_gpt  4785 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.9+ KB


In [71]:
df_gpt_total['run'] = 3

In [72]:
df_gpt_total.tail()

id_gpt sentiment_gpt  run
4780    4780  \n\nNegative    3
4781    4781  \n\npositive    3
4782    4782   \n\nneutral    3
4783    4783  \n\npositive    3
4784    4784   \n\nNeutral    3

In [59]:
#df_gpt_total_all = df_gpt_total.append(df_gpt_total2)

C:\Users\lespinosa\AppData\Local\Temp\ipykernel_23560\3678373193.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gpt_total_all = df_gpt_total.append(df_gpt_total2)


In [73]:
df_gpt_total.to_excel('../data/local/gpt_sentiment_mturk_v4.xlsx')


# Using two categories for vaccine sentiment

In [6]:
tweets_pos_neg = tweets[tweets['label_tag'].isin(['positive', 'negative'])]
tweets_pos_neg.head()

index                   id  \
18     18  1235574994924490752   
22     22  1236451808412909568   
26     26  1237492335430250496   
28     28  1237894709306986496   
38     38  1239077015250030592   

                                                 text label_tag  
18  @user @user I saw #VaxxedII yesterday and I ha...  negative  
22  Dear anti-vaxxers. If you are legitimately wor...  positive  
26  @user @user thank you for the vaccine LOL #Wel...  positive  
28  Remember polio? Killed a lot of people. Vaccin...  positive  
38  You be ready to pay what this Coronavirus vacc...  negative

In [39]:
tweets_new_pos_neg = tweets_new[tweets_new['label_tag'].isin(['positive', 'negative'])]
tweets_new_pos_neg.head()

index                   id  \
1      1  1008751928568840192   
2      2  1008751948957351936   
3      3  1008752099222523904   
4      4  1010156916553736192   
5      5  1010157450618601472   

                                                text label_tag  
1  Great news to see. Hopefully, folks who disput...  positive  
2  The global community must invest in further re...  positive  
3  HPV vaccine has almost wiped out infections in...  positive  
4  So I can catch this shit too and need treatmen...  negative  
5  Could a #vaccine for #type-1 diabetes be on th...  positive

In [42]:
prompt_raw_pos_neg = "Please give me the sentiment regarding vaccination in the following tweet: \
"TWEET_TEXT_HERE\"\nUse one of the following words: negative, positive"

In [40]:
len(tweets_new_pos_neg.index)

128

In [9]:
test = tweets_pos_neg.iloc[0:2, :]
test

index                   id  \
18     18  1235574994924490752   
22     22  1236451808412909568   

                                                 text label_tag  
18  @user @user I saw #VaxxedII yesterday and I ha...  negative  
22  Dear anti-vaxxers. If you are legitimately wor...  positive

In [7]:
tweets_pos_neg1 = tweets_pos_neg.iloc[0:306,:]
tweets_pos_neg2 = tweets_pos_neg.iloc[306:601,:]
tweets_pos_neg3 = tweets_pos_neg.iloc[601:901, :]
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]


In [103]:
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]

In [43]:
id_gpt_two = []
sent_gpt_two = []

In [44]:
for i in range(0,128):
    prompt_two_i = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_new_pos_neg.iloc[i,2])
    response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_i,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt_two.append(tweets_new_pos_neg.iloc[i,0])
    sent_gpt_two.append(response_two.choices[0].text)

In [17]:
prompt_two_1339 = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_pos_neg.iloc[1339,2])
response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_1339,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
id_gpt_two.append(tweets_pos_neg.iloc[1339,0])
sent_gpt_two.append(response_two.choices[0].text)

In [46]:
df_two_gpt_total = pd.DataFrame(list(zip(id_gpt_two, sent_gpt_two)),
                            columns = ['id_gpt_two', 'sentiment_gpt_two'])
df_two_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_gpt_two         128 non-null    int64 
 1   sentiment_gpt_two  128 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [47]:
df_two_gpt_total.tail()

id_gpt_two         sentiment_gpt_two
123         179  , or neutral\n\nPositive
124         180  , or neutral\n\nPositive
125         181  , or neutral\n\nNegative
126         182  , or neutral\n\nPositive
127         184              \n\nNegative

In [108]:
id_gpt13 = id_gpt
sent_gpt13 = sent_gpt
df_gpt_total13 = df_gpt_total

In [25]:
#df_gpt_total1.to_csv('../data/local/gpt_sentiment1.csv')
df_gpt_total1['sentiment_gpt2'] = df_gpt_total1['sentiment_gpt'].to_string()
print(type(df_gpt_total1['sentiment_gpt2']))

<class 'pandas.core.series.Series'>


In [30]:
#sent_gpt1.to_csv('../data/local/gpt_sentiment1_only.csv')

In [48]:
df_two_gpt_total.to_excel('../data/local/gpt_sentiment_new_pos_neg.xlsx')
